# side model

In [1]:
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_4h.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(11332, 3000)
(11332, 7)


16h_dft_dom_cycle_lag23    288
16h_dft_dom_cycle_lag22    284
16h_dft_dom_cycle_lag21    280
16h_dft_dom_cycle_lag20    276
16h_dft_dom_cycle_lag19    272
                          ... 
4h_comb_spectrum_pwr_7       0
4h_hurst_coef_fast           0
4h_highpass_bp               0
4h_ac_39                     0
4h_adaptive_rsi              0
Length: 3000, dtype: int64

In [2]:
df_features = df_features[288:]
label_side = label_side[288:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(11044, 3000)
(11044, 7)


1h_acc_swing_index        0
16h_dual_diff_dt_lag15    0
16h_dft_spectrum_5        0
16h_dft_spectrum_6        0
16h_dft_spectrum_7        0
                         ..
4h_fisher_lag12           0
4h_fisher_lag13           0
4h_fisher_lag14           0
4h_fisher_lag15           0
16h_williams_r_lag9       0
Length: 3000, dtype: int64

In [3]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(11044, 3000)
(array([0, 1]), array([4676, 6368]))
(array([0, 1]), array([ 964, 1245]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-26 00:17:01,119] A new study created in memory with name: no-name-0c8eb372-f647-4887-abc2-c221d43baf56
[I 2025-03-26 00:17:03,748] Trial 0 finished with value: 0.7911379959672715 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 120, 'max_depth': 941, 'min_gain_to_split': 0.4740711716745862, 'min_data_in_leaf': 107, 'lambda_l1': 89.69229494565586, 'lambda_l2': 61.03336636214912, 'num_boost_round': 137}. Best is trial 0 with value: 0.7911379959672715.


precision: 0.7607505863956215, recall: 0.7815261044176707, f1: 0.7709984152139461


[I 2025-03-26 00:17:17,216] Trial 1 finished with value: 0.8064973587295239 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 31, 'max_depth': 646, 'min_gain_to_split': 0.7724771085330039, 'min_data_in_leaf': 486, 'lambda_l1': 96.65249724789983, 'lambda_l2': 58.81045626937072, 'num_boost_round': 1700}. Best is trial 1 with value: 0.8064973587295239.


precision: 0.7858306188925082, recall: 0.7751004016064257, f1: 0.7804286291953093


[I 2025-03-26 00:17:35,467] Trial 2 finished with value: 0.801044843273509 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 73, 'max_depth': 601, 'min_gain_to_split': 0.13848727470070804, 'min_data_in_leaf': 491, 'lambda_l1': 24.711598642440617, 'lambda_l2': 90.3514674256512, 'num_boost_round': 1773}. Best is trial 1 with value: 0.8064973587295239.


...

precision: 0.7699248120300752, recall: 0.8224899598393575, f1: 0.7953398058252428


[I 2025-03-26 00:25:10,264] Trial 98 finished with value: 0.8118057291406289 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 270, 'max_depth': 109, 'min_gain_to_split': 0.6268997100504917, 'min_data_in_leaf': 434, 'lambda_l1': 40.264765669964426, 'lambda_l2': 61.87318353650344, 'num_boost_round': 195}. Best is trial 57 with value: 0.8217067439884017.


precision: 0.779261586802828, recall: 0.7967871485943775, f1: 0.7879269261318507


[I 2025-03-26 00:25:12,405] Trial 99 finished with value: 0.8159251112333148 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 281, 'max_depth': 85, 'min_gain_to_split': 0.7253650795427027, 'min_data_in_leaf': 408, 'lambda_l1': 47.21518962244531, 'lambda_l2': 64.5903381410096, 'num_boost_round': 148}. Best is trial 57 with value: 0.8217067439884017.


precision: 0.7635687732342007, recall: 0.8248995983935743, f1: 0.7930501930501931


In [5]:
study.best_params

{'is_unbalance': False,
 'extra_trees': True,
 'boosting': 'gbdt',
 'num_leaves': 177,
 'max_depth': 456,
 'min_gain_to_split': 0.2963021126613028,
 'min_data_in_leaf': 432,
 'lambda_l1': 70.71108443346776,
 'lambda_l2': 72.37949587865077,
 'num_boost_round': 713}

In [6]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [7]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8217067439884017


In [9]:
model.save_model("model/model_side.txt")
prod_model.save_model("model/model_side_prod.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_4h.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

16h_dft_dom_cycle_dt_lag37     349
16h_dft_dom_cycle_ddt_lag36    349
16h_dft_dom_cycle_dt_lag33     332
16h_dft_dom_cycle_dt_lag30     320
16h_dft_dom_cycle_dt_lag27     308
                              ... 
4h_comb_spectrum_pwr_21          0
4h_comb_spectrum_pwr_22          0
4h_comb_spectrum_pwr_23          0
4h_comb_spectrum_pwr_26          0
side_model_res                   0
Length: 3000, dtype: int64

In [2]:
df_features = df_features[349:]
meta_label = meta_label[349:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(10983, 3000)
(10983, 7)


1h_ac_43                        0
16h_dual_diff_lag5              0
16h_dual_diff_dt_lag2           0
16h_dual_diff_dt_lag28          0
16h_dual_diff_lag10             0
                               ..
4h_fisher_lag8                  0
4h_fisher_lag9                  0
4h_forecast_oscillator_lag22    0
4h_highpass_bp_ddt_lag24        0
side_model_res                  0
Length: 3000, dtype: int64

In [3]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(10983, 3000)
(array([0, 1]), array([1215, 7571]))
(array([0, 1]), array([ 521, 1676]))


In [4]:
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-26 00:36:11,552] A new study created in memory with name: no-name-cb815f0b-46cf-441e-b825-c069fbdadc3f
[I 2025-03-26 00:36:21,540] Trial 0 finished with value: 0.6856658212181291 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 340, 'max_depth': 169, 'min_gain_to_split': 0.9175049809112257, 'min_data_in_leaf': 224, 'lambda_l1': 62.195783526128494, 'lambda_l2': 47.076605858244434, 'num_boost_round': 773}. Best is trial 0 with value: 0.6856658212181291.


precision: 0.8131083711875405, recall: 0.7476133651551312, f1: 0.7789866335094808, auc: 0.6455011246043272, fbeta: 0.6856658212181291


[I 2025-03-26 00:36:26,434] Trial 1 finished with value: 0.7060535357408849 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 370, 'max_depth': 462, 'min_gain_to_split': 0.8930889856472678, 'min_data_in_leaf': 242, 'lambda_l1': 10.402107439987654, 'lambda_l2': 49.165319904405315, 'num_boost_round': 438}. Best is trial 1 with value: 0.7060535357408849.


precision: 0.7883683360258481, recall: 0.8735083532219571, f1: 0.8287574299462214, auc: 0.6388989413602445, fbeta: 0.7060535357408849


[I 2025-03-26 00:36:31,387] Trial 2 finished with value: 0.701151237649668 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 454, 'max_depth': 549, 'min_gain_to_split': 0.7955182924042891, 'min_data_in_leaf': 80, 'lambda_l1': 6.395361116834471, 'lambda_l2': 63.182983625824846, 'num_boost_round': 249}. Best is trial 1 with value: 0.7060535357408849.


...

precision: 0.7985690698954321, recall: 0.8657517899761337, f1: 0.8308044660750071, auc: 0.6479037925047756, fbeta: 0.7177856007892629


[I 2025-03-26 00:52:39,083] Trial 98 finished with value: 0.705512106894875 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 359, 'max_depth': 945, 'min_gain_to_split': 0.2547546324345134, 'min_data_in_leaf': 228, 'lambda_l1': 15.251475866519069, 'lambda_l2': 71.6712200390044, 'num_boost_round': 1264}. Best is trial 9 with value: 0.7202351476231765.


precision: 0.7877485222998388, recall: 0.8747016706443914, f1: 0.8289510884930732, auc: 0.6328785289900549, fbeta: 0.705512106894875


[I 2025-03-26 00:52:44,537] Trial 99 finished with value: 0.7089165266510761 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 392, 'max_depth': 722, 'min_gain_to_split': 0.40928164395874633, 'min_data_in_leaf': 238, 'lambda_l1': 20.482934915387137, 'lambda_l2': 74.2915806515294, 'num_boost_round': 809}. Best is trial 9 with value: 0.7202351476231765.


precision: 0.8001142204454598, recall: 0.8359188544152745, f1: 0.8176247446746425, auc: 0.6378991658230225, fbeta: 0.7089165266510761


In [5]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 180,
 'max_depth': 253,
 'min_gain_to_split': 0.9734849610884184,
 'min_data_in_leaf': 118,
 'lambda_l1': 5.7422698799191725,
 'lambda_l2': 65.87956615148437,
 'num_boost_round': 891}

In [6]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [7]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("model/model_meta.txt")
prod_model.save_model("model/model_meta_prod.txt")